In [1]:
import os
import numpy as np
import pandas as pd
#os.getcwd()
import folium
from folium import plugins
import geopandas
import geojson
import matplotlib.cm as cm
import webbrowser
import glob

In [2]:
data_input_gpx = 'data_input_gpx' 
data_processed_gpx = 'data_processed_gpx' 
data_geojson = 'data_geojson'
visualize = False
#SI_units = True

dir_work = '/home/craigmatthewsmith/gps_tracks'
os.chdir(dir_work)
#os.getcwd()

#os.path.isfile(gpx_file_temp)
#use_SI = True

In [3]:
# functions
def rgb2hex(c):
    hexc = '#%02x%02x%02x'%(int(c[0]*255), int(c[1]*255), int(c[2]*255))
    return(hexc)


In [4]:
geojson_file_list = glob.glob(os.path.join(data_geojson, '*.geojson'))
n_files = len(geojson_file_list)
print(n_files)                              
#print('found %s files' %(n_files))                              
#geojson_file = os.path.join(data_geojson, '2020-03-22_15-06.geojson')
#os.path.isfile(geojson_file)



18


In [5]:
# set speed unit
use_SI = True
if use_SI:
    speed_unit = '(km/h)'
else:
    speed_unit = '(mph)'

# create new GeoJson objects to reduce GeoJSON data sent to Folium map as layer
f_track = lambda x: {'color': '#FC4C02', 'weight': 5} # show some color...


In [9]:
# set up Folium map
#fmap = folium.Map(tiles = None, prefer_canvas=True, disable_3d=True)

#fmap = folium.Map(tiles='Stamen Terrain', prefer_canvas=True, disable_3d=True)
fmap = folium.Map(tiles='Stamen Terrain', location=[37.862606, -121.978372], zoom_start=10) 
folium.TileLayer(tiles = 'OpenStreetMap', name='OpenStreetMap', show=False).add_to(fmap)
folium.TileLayer(tiles = 'Stamen Terrain', name='Terrain Map', show=True).add_to(fmap)
cmap = cm.get_cmap('jet') # matplotlib colormap






In [10]:
fmap

In [12]:

features_tracks = []
features_elevation = []
features_speed = []
f = 0
#for f in range(0, n_files, 1):
for f in range(0, 3, 1):
    geojson_file = geojson_file_list[f]
    print('processing f %s of %s ' %(f, n_files))
    
    # read geojson file
    with open(geojson_file, 'r') as file:
        geojson_data = geojson.load(file)

    #geojson_data

    for feature in geojson_data['features']:
        line = geojson.LineString(feature['geometry']['coordinates'])
        features_tracks.append(geojson.Feature(geometry=line))

    cmin_elevation = min(feature['properties']['elevation'] for feature in geojson_data['features'])
    cmax_elevation = max(feature['properties']['elevation'] for feature in geojson_data['features'])
    f_elevation = lambda x: {'color': rgb2hex(cmap((x['properties']['elevation']-cmin_elevation)/(cmax_elevation-cmin_elevation))), 'weight': 5} # cmap needs normalized data
    t_elevation = folium.features.GeoJsonTooltip(fields=['elevation'], aliases=['Elevation (m)'])

    for feature in geojson_data['features']:
        line = geojson.LineString(feature['geometry']['coordinates'])
        elevation = feature['properties']['elevation']
        features_elevation.append(geojson.Feature(geometry=line, properties={'elevation': elevation}))

    cmin_speed = min(feature['properties']['speed'] for feature in geojson_data['features'])
    cmax_speed = max(feature['properties']['speed'] for feature in geojson_data['features'])
    f_speed = lambda x: {'color': rgb2hex(cmap((x['properties']['speed']-cmin_speed)/(cmax_speed-cmin_speed))), 'weight': 5} # cmap needs normalized data
    t_speed = folium.features.GeoJsonTooltip(fields=['speed'], aliases=['Speed '+speed_unit])

    for feature in geojson_data['features']:
        line = geojson.LineString(feature['geometry']['coordinates'])
        speed = feature['properties']['speed']
        features_speed.append(geojson.Feature(geometry=line, properties={'speed': speed}))


geojson_data_elevation = geojson.FeatureCollection(features_elevation)
folium.GeoJson(geojson_data_elevation, style_function=f_elevation, tooltip=t_elevation, name='Elevation (m)', show=False, smooth_factor=3.0).add_to(fmap)
        
geojson_data_speed = geojson.FeatureCollection(features_speed)
folium.GeoJson(geojson_data_speed, style_function=f_speed, tooltip=t_speed, name='Speed '+speed_unit, show=False, smooth_factor=3.0).add_to(fmap)

geojson_data_track = geojson.FeatureCollection(features_tracks)
folium.GeoJson(geojson_data_track, style_function=f_track, name='Track only', show=True, smooth_factor=3.0).add_to(fmap)

fmap

processing f 0 of 18 
processing f 1 of 18 
processing f 2 of 18 


In [13]:
# add layer control widget
folium.LayerControl(collapsed=False).add_to(fmap)

# save map to html file
fmap.fit_bounds(fmap.get_bounds())

html_file = os.path.join(dir_work, 'index.html')
if os.path.isfile(html_file):
    os.system('rm -f '+html_file)
fmap.save(html_file)
# open html file in default browser
webbrowser.open(html_file, new=2, autoraise=True)


True

In [ ]:


#folium.Marker([45.3288, -121.6625], popup='<i>Mt. Hood Meadows</i>', tooltip=tooltip).add_to(m)
#folium.Marker([45.3311, -121.7113], popup='<b>Timberline Lodge</b>', tooltip=tooltip).add_to(m)

#folium.Circle(
#    radius=100,
#    location=[45.5244, -122.6699],
#    popup='The Waterfront',
#    color='crimson',
#   fill=False,
#).add_to(m)




In [ ]:
for n in range(0, n_points, 1):
    #folium.Marker([lat_array[n], lon_array[n]], popup='<i>temp1</i>', tooltip=tooltip).add_to(m)
    #folium.Marker([lat_array[n], lon_array[n]], tooltip=tooltip).add_to(m)
    # 100
    folium.Circle(
        radius=10,
        location=[lat_array[n], lon_array[n]],
        color='red',
        fill=True,
    ).add_to(m)
    
